In [17]:
import pandas as pd
import numpy as np
from datetime import timedelta
from pandas_ta.momentum import rsi as ta_rsi
import plotly.graph_objects as go


import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 




C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [18]:
klines_file  = './backtest/klines/2d01/Full_BTCUSDT_2d01_2019-01-01_2024-10-15.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #df.set_index('datetime',inplace=True)

#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)

In [19]:
def HeikinAshi(df):
    df['HA_close'] = (df['close'].shift(1) + df['close']) / 2
    df['HA_open'] = (df['open'].shift(1) + df['close'].shift(1)) / 2
    df['HA_high'] = df[['high', 'HA_open', 'HA_close']].max(axis=1)
    df['HA_low'] =  df[['low', 'HA_open', 'HA_close']].min(axis=1)
    df['HA_side'] = np.where(df['HA_close']>df['HA_open'],1,-1)

    #Analizando tipo de vela
    df['HA_side'] = np.where(df['HA_close']>df['HA_open'],1,-1)
    df['HA_cuerpo'] = abs(df['HA_close'] - df['HA_open'])
    df['HA_sombra_inferior'] = df[['HA_open', 'HA_close']].min(axis=1) - df['HA_low']
    df['HA_sombra_superior'] = df['HA_high'] - df[['HA_open', 'HA_close']].max(axis=1)
    condicion_indecision = (
        (df['HA_cuerpo'] <= 0.1 * (df['HA_sombra_inferior'] + df['HA_sombra_superior'])) &  # Cuerpo pequeño
        (df['HA_sombra_inferior'] >= 2 * df['HA_cuerpo']) &  # Sombra inferior larga
        (df['HA_sombra_superior'] >= 2 * df['HA_cuerpo'])    # Sombra superior larga
    )
    df['HA_vela_indecision'] = condicion_indecision
    df['HA_side'] = np.where(df['HA_vela_indecision'],0,df['HA_side'])
    df['HA_low_trend'] = np.where(df['HA_vela_indecision'],1,None)
    df.drop(['HA_vela_indecision','HA_cuerpo','HA_sombra_inferior','HA_sombra_superior'], axis=1, inplace=True)
    return df

In [ ]:
df_origin = df.copy()
# Crear una copia para trabajar en Heikin-Ashi con el close promedio entre close anterior y actual
df = HeikinAshi(df)

df['trend-chg'] = np.where(df['HA_low_trend'],df['close'],None)

#df['close'] = df['HA_close']
#df['open'] = df['HA_open']
#df['high'] = df['HA_high']
#df['low'] = df['HA_low']

df['HA_sl'] =  np.where((df['HA_side']>=0)  & (df['HA_side'].shift(1)>=0),df['HA_low'].shift(2),None)
df['HA_tp'] =  np.where((df['HA_side']<=0) & (df['HA_side'].shift(1)<=0),df['HA_high'].shift(2),None)

#FFILL 
#df.loc[df['HA_tp'].isnull(), 'HA_sl'] = df['HA_sl'].ffill()
#df['HA_sl'] = np.where(df['HA_sl']<df['HA_low'],df['HA_sl'],None)
df['HA_sl'] = np.where((df['HA_sl'].isnull()) & (df['close'])>df['open'],df['HA_sl'].shift(1),None)

df['buy'] = np.where((df['HA_sl'].notnull()) & (df['HA_sl'].shift(1).isnull()),df['close'],None)
df['sell'] = np.where((df['HA_sl'].isnull()) & (df['HA_sl'].shift(1).notnull()),df['close'],None)


In [25]:
df_to_show = df
indicators = [
     {'col': 'HA_sl','name': 'HA_sl','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'HA_tp','name': 'HA_tp','color': 'green','row': 1, 'mode':'markers',},
     {'col': 'trend-chg','name': 'trend-chg','color': 'yellow','row': 1, 'mode':'markers',},
     #{'col': 'st_high','name': 'ST High','color': 'green','row': 1, 'mode':'markers',},
     #   {'col': 'FL_0.0%', 'name': 'FL_0.0%', 'color': 'red', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold1', 'name': 'FL_gold1', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold2', 'name': 'FL_gold2', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_100.0%', 'name': 'FL_100.0%', 'color': 'green', 'row': 1, 'mode': 'lines', }, 
        ]

indicators_out = [
     #{'col': 'sign_acum','name': 'sign_acum','color': 'yellow','row': 1, 'mode':'lines',},
     #{'col': 'VOL_perc','name': 'Volume_SMA','color': 'green','row': 1, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       #{'df':df,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       #{'df':df,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()